In [1]:
import sys
from torch.nn.utils.rnn import pad_sequence
# sys.path.append("./Incontext-learning") # this part works for goole colab

In [2]:
import utility
import metric
import importlib
importlib.reload(utility)
importlib.reload(metric)
from utility import (data_selection, evaluate_zeroshot,evaluate_finetuning,evaluate_demonstrations as  evaluate_fewshots, finetune_model_eval)
from metric import Rec2FTP, SimAOU
from torch.utils.data import DataLoader


In [3]:
import transformers
import torch

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# check point

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset

# Load SST-2 dataset
# dataset = load_dataset("glue", "sst2")

# load SST-5 dataset
dataset = load_dataset("SetFit/sst5")

# Load Qwen3 tokenizer and model
model_name = "Qwen/Qwen3-0.6B"
model_path = "/home/students/wli/UniHeidelberg/semster2/final_projects/models/Qwen3-0.6B-Base"
# model_path = model_name
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map = "auto")

tokenizer.padding_side = "left"
# Make sure tokenizer has pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# if getattr(model.config.pad_token_id, None) is None:
    # pass

Repo card metadata block was not found. Setting CardData to empty.


In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 8544
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 1101
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 2210
    })
})

In [7]:
! nvidia-smi

Sat Sep 20 02:02:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti     Off |   00000000:3B:00.0 Off |                  N/A |
| 25%   39C    P5             15W /  250W |    1295MiB /  11264MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:
train_dataset = dataset["train"] # the orginal paper use this
test_dataset = dataset["validation"]

def add_idx(example, idx):
    example["idx"] = idx
    return example

test_dataset = test_dataset.map(add_idx, with_indices=True)
# Format examples as causal LM inputs
def preprocess_function(examples):
    label_map = {0: "negative", 1: "positive"}
    inputs = [
        f"Sentence: {sentence} Label: {label_map[label]}"
        for sentence, label in zip(examples["sentence"], examples["label"])
    ]
    # Tokenize with padding/truncation
    tokenized = tokenizer(
        inputs,
        truncation=True,
        padding=False,
        max_length=256,
        # return_tensors="pt"
        return_tensors=None
    )
    # Set labels equal to input_ids for causal LM loss
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

def preprocess_function_sst5(examples):
    label_map = {0: "terrible", 1: "bad", 2: "neutral", 3: "good", 4: "very great"}
    inputs = [
        f"Sentence: {sentence} Label: {label_map[label]}"
        for sentence, label in zip(examples["text"], examples["label"])
    ]
    # Tokenize with padding/truncation
    tokenized = tokenizer(
        inputs,
        truncation=True,
        padding=False,
        max_length=256,
        # return_tensors="pt"
        return_tensors=None
    )
    # Set labels equal to input_ids for causal LM loss
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized


# Tokenize the dataset
# tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)
train_tokenized_datasets = train_dataset.map(preprocess_function_sst5, batched = True, remove_columns = train_dataset.column_names)
test_tokenized_datasets = test_dataset.map(preprocess_function_sst5, batched=True, remove_columns=test_dataset.column_names)
# Data collator (handles padding dynamically in batch)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [9]:
# test the behavior of LLM before fine tuing
prompt = "Sentence: a stirring , funny and finally transporting re-imagining of beauty and the beast and 1930s horror films. Label:"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
# You can control max_length or early stopping
output_ids = model.generate(
    **inputs,
    max_new_tokens=1,    # adjust based on expected label length
    do_sample=False,       # greedy decoding
    pad_token_id=tokenizer.pad_token_id
)

# Decode generated tokens
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(generated_text)

Sentence: a stirring , funny and finally transporting re-imagining of beauty and the beast and 1930s horror films. Label: "


In [10]:
# test_dataset = test_dataset.select(range(32)) # use a smaller test set for debugging

In [11]:
for batch in DataLoader(test_dataset, batch_size=2, shuffle=True):
    print(batch)
    print(type(batch))
    break

{'text': ["although there are several truly jolting scares , there 's also an abundance of hackneyed dialogue and more silly satanic business than you can shake a severed limb at .", "it believes it 's revealing some great human truths , when , in reality , it 's churning ground that has long passed the point of being fertile ."], 'label': tensor([2, 2]), 'label_text': ['neutral', 'neutral'], 'idx': tensor([836, 911])}
<class 'dict'>


In [ ]:

a = SimAOU(model,tokenizer,train_dataset, test_dataset, best_seed = 0, num_data_points = 32, data_type = "sst5", batch_size_icl = 32, batch_size_zsl = 16)

  0%|          | 0/138 [00:00<?, ?it/s]

RAM usage: 2303.54 MB


  1%|          | 1/138 [00:09<21:39,  9.48s/it]

RAM usage: 2333.88 MB


  1%|▏         | 2/138 [00:18<20:37,  9.10s/it]

...

 25%|██▌       | 35/138 [05:16<15:21,  8.94s/it]

RAM usage: 8030.00 MB


 26%|██▌       | 36/138 [05:25<15:12,  8.95s/it]

RAM usage: 8030.02 MB


 27%|██▋       | 37/138 [05:33<15:01,  8.92s/it]

In [ ]:
a

In [ ]:
# finetune_model_eval(model, tokenizer,train_dataset,test_dataset, best_seed = 50, data_type="sst5", num_data_points = 32)